In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import torch
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
#device=xm.xla_device()

In [ ]:
device

In [ ]:
import zipfile
z=zipfile.ZipFile('/kaggle/input/movie-review-sentiment-analysis-kernels-only/train.tsv.zip')
train = pd.read_csv(z.open('train.tsv'),delimiter='\t',usecols = ['Phrase','Sentiment'])
z=zipfile.ZipFile('/kaggle/input/movie-review-sentiment-analysis-kernels-only/test.tsv.zip')
test = pd.read_csv(z.open('test.tsv'),delimiter='\t',usecols = ['PhraseId','Phrase'])
sns.countplot(train["Sentiment"])

In [ ]:
import transformers
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig,BertModel
print('transformers version :', transformers.__version__)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from transformers import BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels = 5,output_attentions = False, output_hidden_states = False)
print(model)
config = BertConfig.from_pretrained('bert-base-uncased')
print(config)

In [ ]:
sentences = train.Phrase.values
labels = train.Sentiment.values

In [ ]:
tsentences = test.Phrase.values

In [ ]:
tPhraseId = test.PhraseId.values

In [ ]:
tPhraseId[0]

In [ ]:
tsentences[0]

In [ ]:
max_len = []
for sent in sentences:
    token_words = tokenizer.tokenize(sent)
    max_len.append(len(token_words))
print(max(max_len))

In [ ]:
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        truncation=True,
                        add_special_tokens = True,
                        max_length = 80,
                        padding='max_length',
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                   )

    # 単語ID変換    
    input_ids.append(encoded_dict['input_ids'])
    # Attentionmask
    attention_masks.append(encoded_dict['attention_mask'])
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
# tenosor変換
labels = torch.tensor(labels)

In [ ]:
testinput_ids = []
testattention_masks = []

for sent in tsentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        truncation=True,
                        add_special_tokens = True,
                        max_length = 80,
                        padding='max_length',
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                   )
  
    testinput_ids.append(encoded_dict['input_ids'])
    testattention_masks.append(encoded_dict['attention_mask'])
testinput_ids = torch.cat(testinput_ids, dim=0)
testattention_masks = torch.cat(testattention_masks, dim=0)

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセット
dataset = TensorDataset(input_ids, attention_masks, labels)
testdataset = TensorDataset(testinput_ids, testattention_masks)

train_size = int(len(dataset))
val_size = int(len(testdataset))

print('トレーニングデータ：{}'.format(train_size))
print('テストデータ:　{} '.format(val_size))

# データローダー
batch_size = 32
train_dataloader = DataLoader(dataset,sampler = RandomSampler(dataset),batch_size = batch_size)
validation_dataloader = DataLoader(testdataset,sampler = SequentialSampler(testdataset),batch_size = batch_size)

In [ ]:
model.cuda()

In [ ]:
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

In [ ]:
def train(model):
    model.train() 
    train_loss = 0
    k=0
    #print(k)
    for batch in train_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        #loss, logits 
        a = model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
        loss=a[0]
        logit=a[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
        k=k+1
        ##if k % 100==0:
            ##print(k)
    return train_loss

In [ ]:
max_epoch = 2
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    test_ = train(model)
    train_loss_.append(train_)
    test_loss_.append(test_)

In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
preds=[]
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    with torch.no_grad():      
        pred=model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask)
        preds.append(pred[0])

In [ ]:
yos=[]
yola=[]
for h in preds:
    for hh in h:
        #print(hh)
        y=hh.to('cpu').detach().numpy().copy()
        yos.append(y)
        yola.append(np.argmax(y))

In [ ]:
yola[0]

In [ ]:
id_df = pd.DataFrame(tPhraseId, columns=['PhraseId'])
pred_df = pd.DataFrame(yola, columns=['Sentiment'])
Submission_df = pd.concat([id_df, pred_df], axis=1)
Submission_df.head()

In [ ]:
Submission_df.to_csv('submission.csv', index=False)